In [2]:
#@title 1. Montar drive (If you are in in colab){ display-mode: "form" }

import os
from google.colab import drive
path =  "/content/drive/My Drive/"#@param {type:"string"}
drive.mount('/content/drive')
os.chdir(path)
print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [3]:
#@title 2. Load libraries { display-mode: "form" }
import numpy as np
import sys
import matplotlib.pyplot as plt
from sklearn import datasets

## 3. Descargar datos

In [ ]:
!wget  -P "ircad-dataset.zip" "https://uc024e81de1cd9a40ec056168242.dl.dropboxusercontent.com/cd/0/get/AzG1V_j7NQLUjCMoBFA1AfT1vUVSmQtW6f8jw-hQSjeL7wTXsjCeXQ27YcUAmzvhVR6q-Aq2rwlavnDgvQgldD0KuCTLX-kGG6sFeRU7QrNtAMj7BoqeNlcb3y0pNU6uSI8/file?_download_id=804590774976771846891646756850447922649186922453831301734042422247&_notify_domain=www.dropbox.com&dl=1"

>**Nota:** Omitir la descarga si ya tiene el archivo **ircad-dataset.zip**

In [ ]:
!unzip 'ircad-dataset.zip'

**Importante:** Una vez descomprimida la data encontrara una carpeta llamada **ircad-dataset** dentro se encuentran los 20 conjunto de imagenes donde cada conjunto de imagenes representa a un paciente al que le realizaron la resonancia magnética, ademas encontrara 20 conjuntos de imagenes mas que representan las mascaras o donde se encuentra el higado en la imagen original

Se distinguen de la siguiente forma:

- ircad_e14**_orig**.nii.gz  
Hace referencia al escaneo original sin ninguna modificacion o alteracion.
- ircad_e14**_liver**.nii.gz  
Hace referencia a la mascara donde a partir de la imagen original se señalo unicamente donde se encuentra el higado.

**Pasos antes de ejecutar el notebook.ipynb**  
1. Cree una carpeta llamada **raw**
2. Dentro de **raw** cree dos carpetas que llamara **train** y **test**
3. Divida el conjunto de datos que se encuentra en la carpeta **ircad-dataset** seleccionando y pasando 13 muestras para el **train** y 7 muestras para el **test**. Recuerde en las carpetas tanto de train como test debe encontrarse la imagen original (ej: ircad_e14**_orig**.nii.gz) como su mascara (ej:ircad_e14**_liver**.nii.gz)

## 4. Tratamiento de datos

## 5. Referencias

- [[++](https://www.google.com)]
- [[++](https://www.google.com)]
- [[++](https://www.google.com)]
- [[++](https://www.google.com)]
- [[++](https://www.google.com)]